# Design Resources for Developers

This notebook includes the code used to scrape the resources from a repo forked from __[Brad Traversy](https://twitter.com/traversymedia)__'s curated list, __[design-resources-for-developers](https://github.com/bradtraversy/design-resources-for-developers)__ and is the final project by __[Andrew J Hughes](https://twitter.com/ajhughesdev)__ for __[Code Louisville](https://codelouisville.org)__'s Data Analytics Course 1 Jan 2023 cohort.


In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

import platform
platform.python_version()

#### Scrape the Resources

In [ ]:
try:
    import scrapy
except:
    %pip install scrapy
    import scrapy
from scrapy.crawler import CrawlerProcess

In [ ]:
import json


class JsonWritePipeline(object):
  
    def open_spider(self, spider):
        self.file = open('resourceresults.jl', 'w')
    
    def close_spider(self, spider):
        self.file.close()
    
    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

In [ ]:
import logging


class ResourceSpider(scrapy.Spider):
    name = "resources"
    start_urls = [
        'https://github.com/ajhughesdev/design-resources-for-developers',
    ]
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWritePipeline': 1},
        'FEEDS': {'resourceresults.jl': {'format': 'jl', 'overwrite': True}},
    }
  
    def parse(self, response):
        for category in response.xpath('//h2'):
            category_name = category.xpath('string(.)').extract_first().strip()
            resources = []
            for resource in category.xpath('./following-sibling::table[1]//tr'):
                resource_name_elem = resource.xpath('./td[1]/a/text()')
                if not resource_name_elem:
                    continue
                resource_name = resource.xpath('./td[1]/a/text()').extract_first().strip()
                resource_link = resource.xpath('./td[1]/a/@href').extract_first().strip()
                resource_desc = resource.xpath('./td[2]/text()').extract_first().strip()
                resource_category = category_name
                resources.append({'name': resource_name, 'link': resource_link, 'desc': resource_desc, 'category': resource_category})
                
            yield {'category': category_name, 'resources': resources}

In [ ]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(ResourceSpider)
process.start()

#### Verify the results

In [ ]:
ll resourceresults.jl

In [ ]:
!tail -n 2 resourceresults.jl

#### Analyze and clean the results

In [ ]:
try:
  import pandas as pd
except:
  %pip install pandas
  import pandas as pd
  
df = pd.read_json('resourceresults.jl', lines=True)
df

In [ ]:
# Remove rows with empty resources
df = df[df['resources'].map(len) > 0]

# Remove category 'Table of Contents'
df = df[df['category'] != 'Table of Contents']
# Remove anti-vaxxer facists
df = df[df['resources'].apply(lambda x: 'GetAvataaars' not in [r['name'] for r in x])]

df.to_json('resourceresults.jl', orient='records')

df.head()

#### Save cleaned data to a db

In [ ]:
import psycopg2
from dotenv import load_dotenv
import os

load_dotenv()

# connect to postgresql database
conn = psycopg2.connect(database="resourcesdb", user=os.environ.get('PGUSERNAME'), password=os.environ.get('PGPASSWORD'), host="localhost", port="5432")

c = conn.cursor()
c.execute('''
    CREATE TABLE IF NOT EXISTS design_resources_for_devs_resource (
        id BIGSERIAL PRIMARY KEY,
        name VARCHAR(150) NOT NULL,
        "desc" TEXT NOT NULL,
        link VARCHAR(200) NOT NULL,
        category VARCHAR(150) NOT NULL,
        CONSTRAINT unique_name_category UNIQUE (name, category)
    )''')

for index, row in df.iterrows():
    for resource in row['resources']:
        c.execute("""
            INSERT INTO design_resources_for_devs_resource (id, name, "desc", link, category)
            VALUES (DEFAULT, %s, %s, %s, %s)
            ON CONFLICT ON CONSTRAINT unique_name_category DO NOTHING
            """,
            (resource['name'], resource['desc'], resource['link'], row['category']))

conn.commit()
conn.close()
